In [0]:
tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@learnbucket.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

silver_data = f"{silver_adls}earthquake_data_silver/"

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

In [0]:
start_date = date.today() - timedelta(days=1)
end_date = date.today()

In [0]:
# Catch the data from bronze layer
bronze_output = dbutils.jobs.taskValues.get(taskKey= "Bronze", key="bronze_output")
silver_data = dbutils.jobs.taskValues.get(taskKey= "Silver", key="silver_output")

# Access individual variables
start_date = bronze_output.get("start_date", "")
silver_adls = bronze_output.get("silver_adls", "")
gold_adls = bronze_output.get("gold_adls", "")

print(f"Start Date: {start_date}, Gold ADLS: {gold_adls}")

In [0]:
from pyspark.sql.functions import col, when, udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg
from datetime import date, timedelta


In [0]:
df = spark.read.parquet(silver_data).filter(col("time") > start_date)

In [0]:
df = df.limit(10)

In [0]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.
    """
    try:
        coord = (float(lat), float(lon))
        result = rg.search(coord)[0].get('cc')
        print(f"Processed coordinates: {coord} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {coord} -> {str(e)}")
        return None
        

In [0]:
# register the UDF
get_country_code_udf = udf(get_country_code, StringType())

In [0]:
df_with_location = \
    df.\
        withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

In [0]:
df_with_location_sig_class = \
    df_with_location.\
        withColumn("sig_class",
                   when(col("sig") < 100, "minor").\
                       when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                           otherwise("High") 
                   )

In [0]:
df_with_location_sig_class.head()

In [0]:
# Save the transformed DataFrame to the Gold container
gold_output_path = f"{gold_adls}earthquake_events_gold/"

In [0]:
# Append DataFrame to gold container in Parquet format
df_with_location_sig_class.write.mode("append").parquet(gold_output_path)